In [ ]:
### from keras import applications
from keras import Sequential
from keras.layers.core import Dense, Activation


In [2]:
model = applications.VGG16(weights='imagenet', include_top=True)

In [3]:
model.layers[-1].get_config() # last layer - see units: this had 1000 neurons

{'activation': 'softmax',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'name': 'predictions',
 'trainable': True,
 'units': 1000,
 'use_bias': True}

In [4]:
model.layers.pop() # remove last layer

In [5]:
model.layers

In [6]:
model.layers[-1].get_config()

{'activation': 'relu',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'name': 'fc2',
 'trainable': True,
 'units': 4096,
 'use_bias': True}

In [7]:
top_model = Sequential()
top_model.add(Dense(10,input_shape=(4096,)))
top_model.add(Activation('softmax'))

In [8]:
# CREATE AN "REAL" MODEL FROM VGG16
# BY COPYING ALL THE LAYERS OF VGG16
new_model = Sequential()
for l in model.layers:
    new_model.add(l)

In [9]:
# CONCATENATE THE TWO MODELS
new_model.add(top_model)

In [10]:
# Freeze top layers by setting .trainable = False (freeze the weights)
for layer in new_model.layers[:-1]:
    layer.trainable = False

In [11]:
print (new_model.layers[-2].get_config())
print()
print (new_model.layers[-1].get_config())

{'bias_constraint': None, 'activation': 'relu', 'name': 'fc2', 'kernel_regularizer': None, 'kernel_constraint': None, 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'mode': 'fan_avg', 'seed': None, 'distribution': 'uniform', 'scale': 1.0}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'units': 4096, 'bias_regularizer': None, 'activity_regularizer': None, 'trainable': False}

[{'class_name': 'Dense', 'config': {'batch_input_shape': (None, 4096), 'bias_constraint': None, 'kernel_regularizer': None, 'name': 'dense_1', 'dtype': 'float32', 'activation': 'linear', 'kernel_constraint': None, 'use_bias': True, 'activity_regularizer': None, 'units': 10, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'bias_regularizer': None, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'mode': 'fan_avg', 'seed': None, 'distribution': 'uniform', 'scale': 1.0}}, 'trainable': True}}, {'class_name': 'Activation', 'config': {'activ

In [12]:
from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()


In [13]:
from keras.utils import np_utils

img_rows, img_cols = 32, 32
nb_classes = 10

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols,3)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols,3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

Y_train = np_utils.to_categorical(y_train,nb_classes)
Y_test = np_utils.to_categorical(y_test,nb_classes)

In [14]:
X_train.shape

(50000, 32, 32, 3)

In [15]:
from scipy.misc import imresize
import numpy as np
X_train_resized = np.zeros(shape = (1000,224,224,3))
X_test_resized = np.zeros(shape = (100,224,224,3))

for im_ind in range(1000):
    X_train_resized[im_ind] = imresize(X_train[im_ind],(224,224))
for im_ind in range(100):   
    X_test_resized[im_ind] = imresize(X_test[im_ind],(224,224))

/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/ipykernel/__main__.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/ipykernel/__main__.py:9: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [16]:
new_model.compile(loss='categorical_crossentropy',
          optimizer='sgd',
          metrics=['accuracy'])

new_model.fit(X_train_resized, Y_train[:1000], batch_size=32, 
          nb_epoch=10,verbose=1,
          validation_data=(X_test_resized, Y_test[:100]))

/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1000 samples, validate on 100 samples
Epoch 1/2
1000/1000 [==============================] - 930s 930ms/step - loss: 3.4224 - acc: 0.3400 - val_loss: 1.6244 - val_acc: 0.5600
Epoch 2/2
1000/1000 [==============================] - 937s 937ms/step - loss: 1.3746 - acc: 0.6070 - val_loss: 3.1973 - val_acc: 0.4100


In [17]:
new_model.fit(X_train_resized, Y_train[:1000], batch_size=32, 
          nb_epoch=5,verbose=1,
          validation_data=(X_test_resized, Y_test[:100]))

/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 891s 891ms/step - loss: 1.1436 - acc: 0.6510 - val_loss: 1.6209 - val_acc: 0.5600
Epoch 2/5
1000/1000 [==============================] - 855s 855ms/step - loss: 0.8272 - acc: 0.7450 - val_loss: 3.2309 - val_acc: 0.4400
Epoch 3/5
1000/1000 [==============================] - 856s 856ms/step - loss: 0.7048 - acc: 0.7750 - val_loss: 1.5706 - val_acc: 0.6200
Epoch 4/5
1000/1000 [==============================] - 857s 857ms/step - loss: 0.6932 - acc: 0.7870 - val_loss: 1.0489 - val_acc: 0.6400
Epoch 5/5
1000/1000 [==============================] - 855s 855ms/step - loss: 0.3752 - acc: 0.8780 - val_loss: 0.8969 - val_acc: 0.6800


In [18]:
new_model.fit(X_train_resized, Y_train[:1000], batch_size=32, 
          nb_epoch=5,verbose=1,
          validation_data=(X_test_resized, Y_test[:100]))

/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 1018s 1s/step - loss: 0.3256 - acc: 0.9020 - val_loss: 1.0125 - val_acc: 0.6800
Epoch 2/5
1000/1000 [==============================] - 986s 986ms/step - loss: 0.2658 - acc: 0.9200 - val_loss: 0.9154 - val_acc: 0.7200
Epoch 3/5
1000/1000 [==============================] - 881s 881ms/step - loss: 0.2103 - acc: 0.9430 - val_loss: 1.5816 - val_acc: 0.5300
Epoch 4/5
1000/1000 [==============================] - 988s 988ms/step - loss: 0.1861 - acc: 0.9660 - val_loss: 0.9516 - val_acc: 0.6800
Epoch 5/5
1000/1000 [==============================] - 990s 990ms/step - loss: 0.1780 - acc: 0.9600 - val_loss: 0.9187 - val_acc: 0.7000
